This notebook covers the basic of creating UDTFs for manipulating a `GeoMultiLineString`: A set of one or more associated lines, each of two or more points.

For example: `MULTILINESTRING((0 0, 1 0, 2 0), (0 1, 1 1, 2 1))`

For the API, check the RBC ReadTheDocs page:
* [`Column<GeoMultiLineString>`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.ColumnGeoMultiLineString.html#rbc.heavydb.ColumnGeoMultiLineString)
* [`GeoMultiLineString`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.GeoMultiLineString.html#rbc.heavydb.GeoMultiLineString)

In [14]:
import warnings; warnings.filterwarnings('ignore')

### Connect to the HeavyDB server

In [15]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

GeoMultiLineString requires HeavyDB 7.0 or newer

In [24]:
# NBVAL_IGNORE_OUTPUT
heavydb.version[:3]

(7, 0, 0)

### Load test data

In [17]:
from util import load_test_data
from rbc.tests import _MultiLineStringTestTable
table_name = 'mline_table'
load_test_data(heavydb, _MultiLineStringTestTable, table_name)

List of multilinestrings in `mline_table`

In [18]:
import pandas as pd
descr, result = heavydb.sql_execute(f'select * from {table_name}')
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,ml1,ml2,ml3,ml4
0,"MULTILINESTRING ((1 2,3 4,5 6,7 8,9 10),(2 3,3...","MULTILINESTRING ((0 0,5 0,5 5,0 5),(2 2,2 1,1 ...","MULTILINESTRING ((0 0,6 0,6 6,0 6),(3 3,3 2,2 ...","MULTILINESTRING ((0 0,7 0,7 7,0 7),(4 4,2 4,2 ..."
1,"MULTILINESTRING ((0 0,5 0,5 5,0 5))","MULTILINESTRING ((0 0,6 0,6 6,0 6))","MULTILINESTRING ((0 0,7 0,7 7,0 7))","MULTILINESTRING ((0 0,4 0,4 4,0 4))"
2,"MULTILINESTRING ((1 2,3 4,5 6,7 8,9 10),(3 4,1...","MULTILINESTRING ((0 0,5 0,5 5,0 5),(2 2,2 1,1 ...","MULTILINESTRING ((0 0,6 0,6 6,0 6),(3 3,3 2,2 ...","MULTILINESTRING ((0 0,7 0,7 7,0 7),(4 4,2 4,2 ..."
3,None,None,None,None


### Define a function that operate on GeoMultiLineStrings

Function `to_polygon` takes a `Column<GeoMultiLineString>` as input and construct a polygon.

In [19]:
@heavydb("int32(TableFunctionManager, Column<Z>, OutputColumn<K>)",
         Z=['GeoMultiLineString'], K=['GeoPolygon'], devices=['cpu'])
def to_polygon(mgr, mlinestrings, polygons):
    size = len(mlinestrings)
    mgr.set_output_item_values_total_number(0, mlinestrings.get_n_of_values())
    mgr.set_output_row_size(size)
    # Initialize polygons
    for i in range(size):
        if mlinestrings.is_null(i):
            polygons.set_null(i)
        else:
            coords = mlinestrings[i].to_coords()
            polygons[i].from_coords(coords)
    return size

In [20]:
from pprint import pprint

In [21]:
descr, result = heavydb.sql_execute(f'select ml1 from {table_name}')
pprint(list(result))

[('MULTILINESTRING ((1 2,3 4,5 6,7 8,9 10),(2 3,3 4,1 2))',),
 ('MULTILINESTRING ((0 0,5 0,5 5,0 5))',),
 ('MULTILINESTRING ((1 2,3 4,5 6,7 8,9 10),(3 4,1 2,2 3),(5 6,7 8,9 10))',),
 (None,)]


In [23]:
query = (f'''
    SELECT * FROM TABLE(to_polygon(
        cursor(SELECT ml1 from {table_name})
    ))
''')

descr, result = heavydb.sql_execute(query)
pprint(list(result))

[('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(2 3,3 4,1 2,2 3))',),
 ('POLYGON ((0 0,5 0,5 5,0 5,0 0))',),
 ('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(3 4,1 2,2 3,3 4),(5 6,7 8,9 10,5 6))',),
 (None,)]
